## Two-Tower recommender Model for E-Commerce


In [72]:
import numpy as np 
import pandas as pd 
from IPython.display import display
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)


In [73]:
df_product_embedding = pd.merge(
    pd.read_csv('C:/Users/mprat/Downloads/Amazon Dataset/product_150k.csv'),
    pd.read_parquet('C:/Users/mprat/Downloads/Amazon Dataset/shopping_queries_dataset_products.parquet')[['product_id','product_title']].drop_duplicates(),
    on = ['product_id']
).reset_index(drop=True)

df_product_embedding['pid'] = range(0, df_product_embedding.shape[0])

df_query_embedding = pd.read_csv('C:/Users/mprat/Downloads/Amazon Dataset/query_150k.csv')
df_query_embedding['qid'] = range(0, df_query_embedding.shape[0])

In [74]:
df_query_embedding.head()

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q24,q25,q26,q27,q28,q29,q30,q31,query,qid
0,0.016756,-0.072765,0.010259,0.048307,0.074665,0.084850,0.022835,-0.066671,0.039883,-0.027148,...,0.011363,0.015675,0.018649,0.025668,0.000701,-0.006060,-0.078536,-0.026082,child proof cabinet locks,0
1,-0.007341,-0.010955,0.059960,0.071282,0.022258,0.032136,0.042009,-0.041242,-0.009459,-0.017514,...,-0.014555,-0.002024,0.062305,0.020665,-0.005689,0.032699,-0.045051,-0.096923,ankle stockings for women sheer,1
2,0.027613,0.019582,-0.032154,0.018755,0.037383,-0.025616,0.024898,-0.025434,-0.047236,0.022970,...,-0.007898,-0.011111,0.059078,0.020216,0.017000,-0.034270,-0.007410,-0.026015,gluten free snacks,2
3,0.021799,-0.018171,0.005133,0.049379,0.021239,-0.004974,0.022700,-0.024240,-0.014328,0.018335,...,0.003084,-0.041141,0.033879,-0.002729,-0.015864,0.026388,-0.021622,-0.083685,hair geow,3
4,0.036940,-0.077912,-0.036468,0.036546,0.121124,0.009458,0.050458,-0.060216,-0.033329,-0.018182,...,0.049342,0.031401,0.065255,0.086359,-0.056535,-0.013196,-0.023676,-0.070366,"1 by one, amplified, outdoor hdtv antenna",4


In [8]:
df_product_embedding.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p153,p154,p155,p156,p157,p158,p159,product_id,product_title,pid
0,0.057728,0.011255,-0.023404,0.092429,-0.020029,0.018735,0.003974,0.008556,-0.005657,0.016547,...,0.006825,0.076608,-0.019186,-0.010564,0.000239,-0.043817,-0.097729,B01A5DU2TA,"Crocs Kids' Handle It Rain Boots , Candy Pink, 8 Toddler",0
1,0.038126,0.004931,-0.022067,0.066682,0.009379,0.017764,0.019343,0.039371,0.001445,0.010448,...,0.021710,0.093479,-0.026546,-0.052332,0.004328,-0.019683,-0.112607,B07346RPKN,"Hatley Kids' Little Classic Rain Boots, Pink & Navy, 8 US Child",1
2,0.036354,0.016651,0.012105,0.096476,0.024649,-0.013194,0.018663,0.024827,-0.021811,-0.019258,...,0.026384,0.075618,0.015224,-0.049442,0.055570,-0.058420,-0.071863,B078TCVZ4V,Outee Rubber Kids Rain Boots,2
3,0.046325,0.016477,-0.022943,0.018492,-0.025954,-0.005175,0.018342,0.041428,-0.006838,-0.008621,...,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07BR3PPDR,Hope & Henry Girls' Red Milano Stitch Cardigan,3
4,0.042861,-0.012059,-0.051115,0.127641,0.026230,0.011335,0.004991,0.008443,0.037345,-0.019857,...,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07H5C4P19,Spring&Gege Youth Solid Full Zipper Hoodies Soft Kids Hooded Sweatshirt for Boys and Girls Size 3-4 Years Red,4


In [75]:
print(df_query_embedding.shape, df_product_embedding.shape)

(70756, 34) (147673, 163)


## Basic Idea
1.Importing Libraries                         
2.Defining Parameters and Preparing for Similarity Search                   
3.Building the Annoy Index with Product Embeddings                         
4.Finalizing the Annoy Index                    
5.Preparing for Similarity Search and Result Formatting                    
6.Setting Column Names for the Results DataFrame                    
7.Performing the Similarity Search and Storing Results                   
8.Formatting Results into a DataFrame


In [76]:
query_tower_input_dim = 32
product_tower_input_dim = (32*5)
query_tower_input_dim, product_tower_input_dim

(32, 160)

## Introducing Annoy for query Tower

In [77]:
q = AnnoyIndex(query_tower_input_dim, 'euclidean')
mp_query_dict = {}

for ix,row in df_query_embedding.iterrows():
    mp_query_dict[row['qid']] = row['query']
    
    key = int(row['qid'])
    vec = list(row[['q'+str(x) for x in list(range(query_tower_input_dim))]])
    q.add_item(key,vec)

In [78]:
q.build(100) 
q.save('query.tree')
q = AnnoyIndex(query_tower_input_dim,  'euclidean')
q.load('query.tree')

True

In [79]:
# Define parameters
top_k = 20
max_queries = 50  # Limit for testing
query_neighbors = []

# Retrieve top_k nearest neighbors for each query and store results in `query_neighbors`
for idx, row in df_query_embedding.iterrows():
    query_text = row['query']
    
    # Get the top_k nearest neighbor IDs (excluding the query itself)
    neighbor_ids = q.get_nns_by_item(row['qid'], top_k + 1)[1:]
    
    # Get neighbor texts using `mp_query_dict` and build the row
    neighbors = [mp_query_dict[neighbor_id] for neighbor_id in neighbor_ids]
    query_neighbors.append([query_text] + neighbors)
    
    # Break after processing max_queries for testing purposes
    if idx >= max_queries - 1:
        break

# Generate column names
column_names = ['query_id'] + [f'nearest_{i+1}' for i in range(top_k)]

# Create the DataFrame and display the top results
df_neighbors = pd.DataFrame(query_neighbors, columns=column_names)
display(df_neighbors.head(max_queries))


,query_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,child proof cabinet locks,child locks for cabinets,cabinet locks,cabinet handles,magnetic child safety cabinet locks,lockable tool bag,ring cleaning pen,kids safety locks for cabinets,latch hook kits,electromagnetic door lock,...,surelock luggage lock,infant floor pad,baby safety locks for cabinets,baby proofing cabinet lock bows,cabinet lock,childrens kitchen set,door child lock,kitchen sink hose attachment,gingham door mat,shower garden hose wand
1,ankle stockings for women sheer,cargo pants for men,short cowboy booties for women,pants for women,dress boots for men,waist chains for women,pencil skirts for women,socks long for women,work boots for men,striped panties for women,...,underwear for men,cat boots for men,dropcloths,cargo pants for women,back slit shirts for women,striped dresses for women,slippers for men,day pants for men,socks for booties women,skirts for women
2,gluten free snacks,gluten free lemon cookie,heartland gluten free pasta,vegan jerky,gluten free bread,cereals sugar free,beneful wet dog food,chewy granola bars,lays hot pickle,pickled food,...,ezekiel english muffins raisin,chocolate covered raisins,smart tart,wrinkle reducing cream,jerky without sugar,gluten free stuffing mix,beef jerky,miltons gluten free crackers,yung gravy,bleaching cream
3,hair geow,arhletic sweatshirts,pink earrinfs,snoogle,mypillow sheets,sheepjes yarn,dr browns bottles,furinno,ziploc,dr browns nipples,...,craycort,gillie suit,calligrpahy,crocs,maraschino cherries,lacross boots men rubber,snapple,kill suit gillie,blue wif,timberlans
4,"1 by one, amplified, outdoor hdtv antenna",lego 14 years and up,lume deodoant for private parts,1.25 drain popper without overflow,over the ear headphones charging dock,work headphones,bar set without shaker,know it all wine,stand for headphones,11. he y treverse matts,...,now or laters candy,(hearing aid not amplifer),wired noise cancelling headphones without deep bass,duralex glasses set,1$xrphone cases,cheap vinyl records under 15 dollars,c9 red and white christmas lights,a19 color changing light bulb,free music with prime,ride on battery powered construction
5,adidas sweats slim,fila sweatpants,galaxy adidas shoes men,adidas pants men slim,adidas soccer cleats black,adidas pants men,adidas soccer cleats,adidas superstar all white,adidas beanie,adidas pants,...,adidas sweatband,mens russell sweatpants,adidas running shoes mens,billie eilish hoodie,adidas shoes men,running jacket women adidas,adidas original superstar women,adidas mens shoes samba,adidas pants women,neck warmer adidas
6,beta sitosterol now,help the beatles mps,hempcrete,liw hisyamine probiotic,zervos,dr dans cortibalm lip balm,amzcool,crystal bags to hold crystals,graco modes to grow,altavoz jbl,...,garmin vivofit band,trp spyre,cool wallet gocci collors,didlo vibarator sex,credit card tool wallet multitool,lansinoh therapearl 3-in-1 hot or cold breast therapy pack,bathing suit surong women,testalize,zometool,bali bras
7,iphone 11 pro,ipad pro 11,ipad pro,screen protector iphone 8,apple ipad pro 11,apple iphone 11 pro unlocked,screen protector iphone 6s,iphone 11 accessories,ipad pro screen protector 11,screen protector iphone x,...,apple iphone 8 case,phone case iphone 8,iphone 11 pro max,phone screen protector iphone 7,iphone xr cases,apple ipad pro,phone case iphone 6,wireless controller ipad pro,iphone 6s,apple iphone 7 case
8,large training pads for dogs,dog beds for large dogs,dog costumes for large dogs,pet beds for large dogs,dog harness for large dogs,water bowls for large dogs,dog clothes for large dogs,mats for exercise,extra large dog beds for german shepherds,shock collar for 2 dogs,...,food bowls for small dogs,heavy jump ropes for exercise training,dog belly bands for male dogs,camera for monitoring do

In [80]:
print(df_product_embedding.shape)

df_product_embedding.head()

(147673, 163)


,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p153,p154,p155,p156,p157,p158,p159,product_id,product_title,pid
0,0.057728,0.011255,-0.023404,0.092429,-0.020029,0.018735,0.003974,0.008556,-0.005657,0.016547,...,0.006825,0.076608,-0.019186,-0.010564,0.000239,-0.043817,-0.097729,B01A5DU2TA,"Crocs Kids' Handle It Rain Boots , Candy Pink, 8 Toddler",0
1,0.038126,0.004931,-0.022067,0.066682,0.009379,0.017764,0.019343,0.039371,0.001445,0.010448,...,0.021710,0.093479,-0.026546,-0.052332,0.004328,-0.019683,-0.112607,B07346RPKN,"Hatley Kids' Little Classic Rain Boots, Pink & Navy, 8 US Child",1
2,0.036354,0.016651,0.012105,0.096476,0.024649,-0.013194,0.018663,0.024827,-0.021811,-0.019258,...,0.026384,0.075618,0.015224,-0.049442,0.055570,-0.058420,-0.071863,B078TCVZ4V,Outee Rubber Kids Rain Boots,2
3,0.046325,0.016477,-0.022943,0.018492,-0.025954,-0.005175,0.018342,0.041428,-0.006838,-0.008621,...,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07BR3PPDR,Hope & Henry Girls' Red Milano Stitch Cardigan,3
4,0.042861,-0.012059,-0.051115,0.127641,0.026230,0.011335,0.004991,0.008443,0.037345,-0.019857,...,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07H5C4P19,Spring&Gege Youth Solid Full Zipper Hoodies Soft Kids Hooded Sweatshirt for Boys and Girls Size 3-4 Years Red,4


## Introducing Annoy for Product Tower

In [81]:
p = AnnoyIndex(product_tower_input_dim, 'euclidean')
mp_product_dict = {}

for ix,row in df_product_embedding.iterrows():
    mp_product_dict[int(row['pid'])] = row['product_title']
    
    key = int(row['pid'])
    vec = list(row[['p'+str(x) for x in list(range(product_tower_input_dim))]])
    
    p.add_item(key,vec)

In [82]:
p.build(100) 
p.save('product.tree')
p = AnnoyIndex(product_tower_input_dim,  'euclidean')
p.load('product.tree')

True

In [84]:
len(mp_product_dict.items())

147673

In [85]:
# Parameters
top_k = 20
max_products = 50  # Limit for testing purposes
product_neighbors = []

# Retrieve top_k nearest neighbors for each product
for idx, row in df_product_embedding.iterrows():
    product_title = row['product_title']
    
    # Get the top_k nearest neighbors (excluding the product itself)
    neighbor_ids = p.get_nns_by_item(row['pid'], top_k + 1)[1:]
    
    # Retrieve titles of nearest neighbors using `mp_product_dict`
    nearest_titles = [mp_product_dict[neighbor_id] for neighbor_id in neighbor_ids]
    product_neighbors.append([product_title] + nearest_titles)
    
    # Stop early if we reach the test limit
    if idx >= max_products - 1:
        break

# Column names: start with 'product_id' and add 'nearest_1' to 'nearest_k'
column_names = ['product_id'] + [f'nearest_{i+1}' for i in range(top_k)]

# Create DataFrame and display results
df_neighbors2 = pd.DataFrame(product_neighbors, columns=column_names)
display(df_neighbors2.head(5))


,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,"Crocs Kids' Handle It Rain Boots , Candy Pink, 8 Toddler","Crocs Kids' Handle It Rain Boots , Candy Pink, 12 Little Kid","Crocs Kids' Classic Clog , Black, 9 Toddler","Crocs Kids' Handle It Rain Boots , Navy, 9 Toddler","Crocs Kids' Crocband Clog , Party Pink, 11 Little Kid","Crocs Kids' Handle It Rain Boots , Navy, 7 Toddler","Crocs Kids' Crocband Clog , Party Pink, 13 Little Kid","Crocs Kids' Classic Clog , Pink Lemonade, 4 Toddler","Crocs Kids' Unicorn Clog, Ballerina Pink/New Mint, 2 Little Kid","Crocs Kids' Handle It Rain Boots , Navy, 12 Little Kid",...,"Crocs Kids' Classic Clog , Lavender, 7 Toddler","Crocs Kids' Classic Clog , Lavender, 3 Little Kid","Crocs Kids' Classic Clog , Lavender, 4 Toddler","Crocs Kids' Classic Clog , Lavender, 5 Big Kid","Crocs Girls' Isabella Sandal Preschool Flat, paradise pink, 12 M US Little Kid","Crocs Crocband Sandal Kids Unisex Niños Sandalias, Azul (Navy/Red), 27/28 EU","Crocs Kids' Crocband Sandal, Navy/Red, 10 M US Toddler","Crocs Kids' Classic Glitter Clog , Black, 10 Toddler","Crocs Unisex Men's and Women's Classic Clog, Electric Pink, 6 US","crocs Bump It Rain Boot (Toddler/Little Kid), Candy Pink/Oyster, 3 M US Little Kid"
1,"Hatley Kids' Little Classic Rain Boots, Pink & Navy, 8 US Child","Hatley unisex child Classic Boots Raincoat, Navy & Yellow, 2 US Youth","Hatley Girls' Toddler Printed Rain Boot, Navy & White Hearts, 6 US Child","Westcott Classic Kids Scissors, Blunt Tip, 5 Inch, Neon Pink (15967)","Merrell Kids' Unisex M-Bare Steps H20 Water Shoe, Pink, 6 Medium US Toddler","Burton Mens Reserve Bib Pant, True Black 1, Medium","Bloch Dance Bunnyhop Ballet Slipper (Toddler/Little Kid) Little Kid (4-8 Years), Pink - 12 B US Little Kid","Plano Fusion Bow Case, Pink, Small","Bloch Dance Bunnyhop Ballet Slipper (Toddler/Little Kid) Little Kid (4-8 Years), Pink - 11 C US Little Kid","Bloch Women's Performa Dance Shoe, Theatrical Pink, 7",...,"Dearfoams Women's Beatrice Side Gore Slide Slipper, Fresh Pink, Large","Capezio Daisy 205 Ballet Shoe (Toddler/Little Kid),Ballet Pink,7 M US Toddler",Pink Motocross Motorcycle Goggles Dirt Bike ATV Racing Mx Goggles for Men Women Youth Kids (C42),"Capezio Little Girls' Hold & Stretch Footed Tight, Ballet Pink, Small","Capezio Daisy 205 Ballet Shoe (Toddler/Little Kid),Ballet Pink,9.5 M US Toddler","Avery Durable View 3 Ring Binder, 1 Inch Slant Rings, 1 Pink Binder (17830)",WINGHOUSE X Minnie 3D Bow Round Crossbody Mini (My First) Purse Small Cross Bag with Adjustable Shoulder Strap for Kids Little Girls (2-4Y),"Clarks womens Sillian Paz Slip On Loafer, Black Synthetic Nubuck, 9.5 US","Merrell womens Siren Edge 3 Hiking Shoe, Navy/Wave, 8.5 US",Falari Baseball Cap Adjustable Size for Running Workouts and Outdoor Activities All Seasons (1pc Black)
2,Outee Rubber Kids Rain Boots,HDE Womens Rainbow Striped Retro Fashion Dolphin Running Workout Shorts (White),"Utopia Towels 100 Pack Commercial Shop Towels - Cleaning Rags, White",Tankee Slip - Black - Large-DD,AWAYTR Realistic Printed Butterfly Hair Clip 12Pcs Fairy Hair Clips Set Hair Accessories for Girls Women (Purple),Dansko Professional Women Mules & Clogs Shoes White Box Size 40,Nike Jumpman Premium 9A1640-681 Laptop Bookbag Basketball Boys Backpack - Gym Red,VIPKID Orange Teaching T-Shirt (Unisex) (L),Reebok Men's Shoes Legacy Lifter II Black Orange Flare True Grey 8 Size 10,goodr RUNNING SUNGLASSES - (Blue w/Blue Lens),...,Baby Boys Kids Bicycle Croatia Flag Cotton Short Sleeve T-Shirt Size 2-6 Pink,"PaddleTek Element Pickleball Paddle, ProPolyCore Honeycomb Composite (red)",Stance Gamut II Super Invisible Sock Large White 3 Pack,"eclife 24"" Black Bathroom Vanities Set (Contemporary/T03 B02)",SoB Flowerpot Treeman Baby Groot Succulent Planter Cute Green Plants Flower Pot G

In [86]:
import gc
gc.collect()

2068

In [88]:
df_dataset = pd.merge(pd.read_csv('C:/Users/mprat/Downloads/Amazon Dataset/dataset_150k.csv'), 
                      df_product_embedding[['product_id','product_title']].drop_duplicates(), on = ['product_id']
                     )

print(df_dataset.shape)
df_dataset.head()

(157097, 197)


,query,product_id,esci_label,split,q0,q1,q2,q3,q4,q5,...,p151,p152,p153,p154,p155,p156,p157,p158,p159,product_title
0,monogram crossbody bags for women,B07SC67L97,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,0.009924,0.015074,0.014845,0.030969,-0.002622,-0.004156,-0.004414,-0.052931,-0.062627,Tommy Hilfiger Jacquard Crossbody Purse With Monogram (Beige)
1,monogram crossbody bags for women,B088BMKTVZ,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,0.022348,0.072437,0.017069,0.044085,0.034383,-0.037070,-0.031683,-0.046816,-0.130665,Montana West Leather Cowhide Tote Bag For Women Western Fringe Handbag Shoulder Bag With Matching Wallet (Z-Coffee3)
2,monogram crossbody bags for women,B07NVTDGVN,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,-0.007226,0.030359,0.017682,0.054370,-0.040309,-0.028475,0.002260,-0.008909,-0.113362,"Tory Burch Women's Gemini Link Canvas Mini Bag, Black, One Size"
3,replicia tory burch bags,B07NVTDGVN,E,train,-0.010582,-0.032959,-0.027428,0.031883,-0.000479,0.062785,...,-0.007226,0.030359,0.017682,0.054370,-0.040309,-0.028475,0.002260,-0.008909,-0.113362,"Tory Burch Women's Gemini Link Canvas Mini Bag, Black, One Size"
4,green advertine crystal,B07H8GJ44Q,E,train,0.025034,-0.025958,-0.021118,0.026299,0.006780,0.022166,...,0.011845,0.031754,0.026041,0.061598,-0.041917,-0.050758,0.015715,-0.050046,-0.078058,"Crystal Glass Diamond Shaped Decoration, Green 60mm Jewel Paperweight,Gift Decoration Idea For Christmas, Thanksgiving (Please identify our brand Yarr Store)"


In [89]:
df_dataset.shape

(157097, 197)

In [90]:
df_dataset.esci_label.value_counts()

esci_label
E    79252
I    77845
Name: count, dtype: int64

In [91]:
df_dataset[df_dataset.esci_label == 'E'][['query','product_title','split']].sample(20)

,query,product_title,split
148765,electrolyte,"Electrolyte Powder: Orange - 90 Servings - Sugar Free Drink Mix- Keto Electrolyte Powder: No Sugar, Gluten Free Hydration Powder- Keto Electrolytes Supplement: Magnesium, Potassium, Calcium",test
51405,rock band stickers and decals,"Band Stickers Pack Rock Roll Stickers -100 Pcs Vinyl Waterproof Stickers for Personalize Laptop, Electronic Organ, Guitar, Piano, Skateboard, Luggage Graffiti Decals",train
27927,boots for women,MUK LUKS Women's Nikki Boots - Grey,train
105438,tools for tire removal,Ken-Tool KEN29999 Valve Capper PRO Cap and Core Remover/Installer (ValvecapperPRO Cap and Core Remover/Installer),train
127107,large absorbent pads,"Amazon Basics Dog and Puppy Pads, Heavy Duty Absorbency Pee Pads with Leak-proof Design and Quick-dry Surface for Potty Training, X-Large (28 x 34 Inches) - Pack of 25",train
143325,carhartt hats for kids,"Carhartt Girls' Little Hat, Realtree Xtra, One Size",train
24235,evening jumpsuits for women,"KOH KOH Plus Size Womens Short Sleeve Sexy Formal Cocktail Casual Cute Long Pants One Piece Fall Pockets Dressy Jumpsuit Romper Long Leg Pant Suit Suits Outfit Playsuit, Black XL 14-16",train
110500,hand soap refill,"JR Watkins Gel Hand Soap Refill Pouch, Ocean Breeze, 6 Pack, Scented Liquid Hand Wash for Bathroom or Kitchen, USA Made and Cruelty Free, 34 fl oz",test
40020,brown sugar,"C&H, Cane Sugar, Dark Brown, 2lb Bag (Pack of 2)",train
134714,white htv,"Siser EasyWeed HTV 11.8"" x 5yd Roll (White) - Iron on Heat Transfer Vinyl for T-Shirts - Works with Cricut and Silhouette Cutters",train


In [92]:
df_dataset[df_dataset.esci_label == 'I'][['query','product_title','split']].sample(20)

,query,product_title,split
890,race ramps,Godfer Arabe. Unisex Tank Top Reagan Bush '84 Baby Onesies Bodysuit Jumpsuit,generated
12633,rx100 vii quick strap,"Purina Fancy Feast Gravy Wet Cat Food Variety Pack, Gravy Lovers Poultry & Beef Feast Collection - 3 oz. Cans (Pack of 24)",generated
147264,valentines shirt,"Knape & Vogt QT50PB-W Replacement Trash Can, 21.56-Inch by 15.55-Inch by 11.13-Inch,White",generated
102225,autism license plate frame metal,Fancy Nancy and the Quest for the Unicorn: Includes Over 30 Stickers!,generated
62331,power beats 3 earbuds replacement tips,"Sorbus Teepee Play Tent for Kids — Includes Portable Carry Bag for Travel or Storage — Great for Playroom, Bedroom, Nursery, and more",generated
99060,sports bedding,Chemical Guys TVD_102_16 New Car Shine Premium Dressing (16 oz),generated
111624,yellow boots for women thigh high,"MOOSOO Electric Gooseneck Kettle with Variable Temperature Control & Presets, Pour Over Coffee/Tea Kettle, 100% Stainless Steel Inner Lid & Bottom,1000W Rapid Heating, 0.8L, Black",generated
76343,toms,"TOMS Women's Classic Canvas Slip-On,Ash,8.5 M US",test
70063,iphone to aux adapter,Guytalk Men's Letterman Style Premium Thick Fabric Varsity Baseball Jacket Large BROWNKHAKI,generated
47923,zen and the art of motorcycle maintenance,"The Alchemist, 25th Anniversary: A Fable About Following Your Dream",train


In [93]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import transformers
from tensorflow.keras.utils import plot_model

In [94]:
print([p for p in df_product_embedding.columns])
print([p for p in df_query_embedding.columns])

['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'p64', 'p65', 'p66', 'p67', 'p68', 'p69', 'p70', 'p71', 'p72', 'p73', 'p74', 'p75', 'p76', 'p77', 'p78', 'p79', 'p80', 'p81', 'p82', 'p83', 'p84', 'p85', 'p86', 'p87', 'p88', 'p89', 'p90', 'p91', 'p92', 'p93', 'p94', 'p95', 'p96', 'p97', 'p98', 'p99', 'p100', 'p101', 'p102', 'p103', 'p104', 'p105', 'p106', 'p107', 'p108', 'p109', 'p110', 'p111', 'p112', 'p113', 'p114', 'p115', 'p116', 'p117', 'p118', 'p119', 'p120', 'p121', 'p122', 'p123', 'p124', 'p125', 'p126', 'p127', 'p128', 'p129', 'p130', 'p131', 'p132', 'p133', 'p134', 'p135', 'p136', 'p137', 'p138

In [95]:
# Convert 'esci_label' column to binary labels
df_dataset['binary_label'] = df_dataset['esci_label'].apply(lambda x: 1 if x == 'E' else 0)

# Split the dataset into training and validation
train_data = df_dataset[df_dataset['split'] != 'test']
val_data = df_dataset[df_dataset['split'] == 'test']

train_labels = np.array(train_data['binary_label'])
val_labels = np.array(val_data['binary_label'])

train_labels = train_labels.astype('float32')
val_labels = val_labels.astype('float32')



print(train_data.esci_label.value_counts(),
      val_data.esci_label.value_counts())

query_tower_cols = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 
                                  'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 
                                  'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 
                                  'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31']

product_tower_cols = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
                      'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20',
                      'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30',
                      'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40',
                      'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50',
                      'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60',
                      'p61', 'p62', 'p63', 'p64', 'p65', 'p66', 'p67', 'p68', 'p69', 'p70',
                      'p71', 'p72', 'p73', 'p74', 'p75', 'p76', 'p77', 'p78', 'p79', 'p80',
                      'p81', 'p82', 'p83', 'p84', 'p85', 'p86', 'p87', 'p88', 'p89', 'p90',
                      'p91', 'p92', 'p93', 'p94', 'p95', 'p96', 'p97', 'p98', 'p99', 'p100',
                      'p101', 'p102', 'p103', 'p104', 'p105', 'p106', 'p107', 'p108', 'p109', 'p110',
                      'p111', 'p112', 'p113', 'p114', 'p115', 'p116', 'p117', 'p118', 'p119', 'p120',
                      'p121', 'p122', 'p123', 'p124', 'p125', 'p126', 'p127', 'p128', 'p129', 'p130',
                      'p131', 'p132', 'p133', 'p134', 'p135', 'p136', 'p137', 'p138', 'p139', 'p140',
                      'p141', 'p142', 'p143', 'p144', 'p145', 'p146', 'p147', 'p148', 'p149', 'p150',
                      'p151', 'p152', 'p153', 'p154', 'p155', 'p156', 'p157', 'p158', 'p159'
                     ]



train_inputs = [
    np.array(train_data[query_tower_cols]),
    
    np.array(train_data[product_tower_cols])
    
]

val_inputs = [
    np.array(val_data[query_tower_cols]),
    
    np.array(val_data[product_tower_cols])
]

esci_label
I    75124
E    61682
Name: count, dtype: int64 esci_label
E    17570
I     2721
Name: count, dtype: int64


In [96]:
import gc
gc.collect()

0

### Importing essential libraries

In [97]:
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten, Dot, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras.layers import Input, Dense, Embedding, Flatten, Lambda
from keras.models import Model
import torch

## Query & Product dimensions are different


In [98]:
print(query_tower_input_dim, product_tower_input_dim)

32 160


## An Two-tower model architecture for Product Search



In [99]:
import tensorflow as tf
from keras.layers import Input, Dense, Dot, Lambda
from keras.models import Model
from keras import backend as K


embedding_dim = 22


# Input layers for tokenized sequences
#Query
input_query = Input(shape=(query_tower_input_dim,), name='input_query')
query_embedding_1= Dense(32, activation='elu', name='embedding_layer_query_1')(input_query)
query_embedding_2= Dense(26, activation='linear', name='embedding_layer_query_2')(query_embedding_1)
final_query_embedding = Dense(embedding_dim, activation='linear', name='embedding_layer_query_3')(query_embedding_2)
normalized_query = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), name='normalize_query')(final_query_embedding)

#Product
input_product = Input(shape=(product_tower_input_dim,), name='input_product')
product_embedding_1 = Dense(48, activation='elu', name='embedding_layer_product_1')(input_product)
product_embedding_2 = Dense(26, activation='linear', name='embedding_layer_product_2')(product_embedding_1)
final_product_embedding = Dense(embedding_dim, activation='linear', name='embedding_layer_product_3')(product_embedding_2)
normalized_product = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), name='normalize_product')(final_product_embedding)

cosine_similarity = Dot(axes=1, normalize=True, name='cosine_similarity')([normalized_product, normalized_query])

# Build your model
model = Model(inputs=[input_query, input_product], outputs=cosine_similarity)


# Summary of the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_product (InputLayer)    │ (None, 160)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_query (InputLayer)      │ (None, 32)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_layer_product_1     │ (None, 48)                │           7,728 │ input_product[0][0]        │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_layer_query_1       │ (None, 32)                │           1,056 │ input_query[0][0]          │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_layer_product_2     │ (None, 26)                │           1,274 │ embedding_layer_product_1… │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_layer_query_2       │ (None, 26)                │             858 │ embedding_layer_query_1[0… │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_layer_product_3     │ (None, 22)                │             594 │ embedding_layer_product_2… │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_layer_query_3       │ (None, 22)                │             594 │ embedding_layer_query_2[0… │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalize_product (Lambda)    │ (None, 22)                │               0 │ embedding_layer_product_3… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalize_query (Lambda)      │ (None, 22)                │               0 │ embedding_layer_query_3[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ cosine_similarity (Dot)       │ (None, 1)                 │               0 │ normalize_product[0][0],   │
│                               │                           │                 │ normalize_query[0][0]      │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 12,104 (47.28 KB)

 Trainable params: 12,104 (47.28 KB)

 Non-trainable params: 0 (0.00 B)

In [112]:
plot_model(model,show_shapes=1,show_layer_activations=1)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [113]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dot, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [114]:
# Define a model checkpoint callback to save the best model weights during training
checkpoint_path = 'best_model_weights.keras' 
model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',  
    verbose=1,
    save_best_only=True,  
    mode='min'  
)

model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error', metrics=['accuracy'])



history = model.fit(

    train_inputs, train_labels, epochs=30, batch_size=64, 
    validation_data=(val_inputs, val_labels),
    callbacks=[model_checkpoint],
    # Giving positive class more importance to positive labels to ensure precision is high.
    class_weight={0: 1, 1: 2} )


Epoch 1/30
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4607 - loss: 0.9833
Epoch 1: val_loss improved from inf to 0.52381, saving model to best_model_weights.keras
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - accuracy: 0.4607 - loss: 0.9833 - val_accuracy: 0.8071 - val_loss: 0.5238
Epoch 2/30
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5022 - loss: 0.9394
Epoch 2: val_loss improved from 0.52381 to 0.50169, saving model to best_model_weights.keras
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.5022 - loss: 0.9394 - val_accuracy: 0.8267 - val_loss: 0.5017
Epoch 3/30
2129/2138 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5866 - loss: 0.8701
Epoch 3: val_loss did not improve from 0.50169
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5866 - loss: 0.8701 - val_accuracy: 0.8045 - val_loss: 0.5314
Epoch 4/30
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5988 - loss: 0.8592
Epoch 4: val_loss did not improve from 0.50169
2138/2138 ━━━

In [115]:
# After training, you can load the best model weights using:
model.load_weights(checkpoint_path)

## Chop-off the product tower & generate/get product embeddings

In [116]:
product_model = Model(inputs=input_product,
                         outputs=model.get_layer('normalize_product').output
                   )

In [117]:
plot_model(product_model)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [118]:
input_data_product = np.array(df_product_embedding[product_tower_cols])
product_embeddings = product_model.predict(input_data_product)
len(product_embeddings)

4615/4615 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step


147673

In [119]:
df_product_embeddings_model = pd.DataFrame(product_embeddings, columns = [f'p{x}'for x in range(embedding_dim)] )

df_product_embeddings_model['product_id'] = df_product_embedding['product_id']
df_product_embeddings_model['product_title'] = df_product_embedding['product_title']
df_product_embeddings_model['pid'] = df_product_embedding['pid']

df_product_embeddings_model.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p15,p16,p17,p18,p19,p20,p21,product_id,product_title,pid
0,-0.140432,-0.133850,-0.112938,-0.065813,0.147024,-0.246022,0.146200,0.294778,-0.194798,0.144220,...,-0.112249,0.032859,0.221958,-0.195427,-0.121119,0.626246,0.013190,B01A5DU2TA,"Crocs Kids' Handle It Rain Boots , Candy Pink, 8 Toddler",0
1,-0.137085,-0.116684,-0.087898,-0.088460,0.130982,-0.240888,0.131865,0.295977,-0.209829,0.149933,...,-0.098439,0.047934,0.189255,-0.182804,-0.119724,0.647879,-0.010072,B07346RPKN,"Hatley Kids' Little Classic Rain Boots, Pink & Navy, 8 US Child",1
2,-0.133927,-0.087343,-0.024182,-0.125487,0.109146,-0.266930,0.105339,0.280643,-0.259706,0.216793,...,-0.047180,0.077564,0.119471,-0.172710,-0.119385,0.616564,-0.042134,B078TCVZ4V,Outee Rubber Kids Rain Boots,2
3,-0.111782,-0.019218,0.008779,-0.192815,0.025183,-0.099404,0.039802,0.265468,-0.199347,-0.095136,...,-0.077797,0.155865,-0.007034,-0.067748,-0.055644,0.743726,-0.183913,B07BR3PPDR,Hope & Henry Girls' Red Milano Stitch Cardigan,3
4,-0.137441,-0.061559,-0.050173,-0.154794,0.062743,-0.105633,0.070941,0.253834,-0.157259,-0.227622,...,-0.125252,0.163225,0.041888,-0.097435,-0.039572,0.716282,-0.180637,B07H5C4P19,Spring&Gege Youth Solid Full Zipper Hoodies Soft Kids Hooded Sweatshirt for Boys and Girls Size 3-4 Years Red,4


#### We can analyse the quality of learnt products embeddings of 22-dimensions by doing an ANN search/similarity search

### Introducing Annoy for trained Product tower

In [120]:
pm = AnnoyIndex(embedding_dim, 'dot')
mp_product_dict = {}

for ix,row in df_product_embeddings_model.iterrows():
    mp_product_dict[int(row['pid'])] = row['product_title']
    
    key = int(row['pid'])
    vec = list(row[['p'+str(x) for x in list(range(embedding_dim))]])
    
#print(key,vec)
    pm.add_item(key,vec)

In [121]:
pm.build(100) 
pm.save('product_model.tree')
pm = AnnoyIndex(embedding_dim,  'dot')
pm.load('product_model.tree')

True

In [122]:
top_k = 20
mat = []
for ix,row in df_product_embeddings_model.iterrows():
    item = row['product_title']
    mat.append([item] + [mp_product_dict[x] for x in pm.get_nns_by_item(row['pid'], top_k+1)[1:]])
    
    if ix == 50:
        break
    
cols = ['product_id']
for i in range(top_k):
    cols += ['nearest_{}'.format(i+1)]

print(cols)

df_neighbors3 = pd.DataFrame(mat, columns = cols)

display(df_neighbors3.head(5))

['product_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,"Crocs Kids' Handle It Rain Boots , Candy Pink, 8 Toddler","Franco Kids Bedding Super Soft Plush Cuddle Pillow Buddy, One Size, Disney Frozen 2 Elsa",Smiffys Christmas Tree Hat,"New Era MLB Boston Red Sox Junior Team Classic Alternative 39Thirty Stretch Fit Cap, Blue, Child/Youth","NFLPA Dog Jersey - Tom Brady #12 Pet Jersey - NFL New England Patriots Mesh Jersey, Medium","adidas Boy's X GHOSTED.4 Soccer Shoe, Solar Yellow/Black/Team Royal Blue, 2 Little Kid","Girls Animals Ear Moving Jumping Hats, Warm Plush Rabbit Winter Caps, Cute Bunny Fuzzy (Koala Blue)",Down To Fiesta - Cinco de Mayo Costume Party Outfit T Shirt,"Crocs Kids' Classic Glitter Clog , Black, 10 Toddler","Long Blue Wig for Girls Women ,Winshope Curly Wavy Natural Hair Wigs for Cosplay Costumes Halloween Party Heat Resistant Synthetic Fiber (Include Wig Cap)",...,Vintage Surely Not Everyone Was Kung Fu Fighting T-Shirt,"Crocs Kids' Classic Clog , Slate Grey, 4 Big Kid",Kids Long Wavy Cosplay Wig with Crown for Halloween Costumes and Party (Dark Root Blue),"adidas Boy's Nemeziz 19.4 Firm Ground Soccer Shoe, Coral/Black/Glory red, 2.5 Little Kid","Synthetic Long Hat Hair Extensions Curly Wavy Corn Wave Hairpiece Baseball Cap With Hair Attached Adjustable Cap Black Hat Wigs Yaki Hair Extensions For Girls Lady Women(16''-Curly,Light Brown)","Fruit of the Loom Boys Big 10 Pack Crew Socks, Black Assorted, Shoe: 3-9","MOHEEN Men's Moisture Wicking Side Blocked Polo's Big & Tall Sizes (3XL,Red)","Tommy Hilfiger Men's Ardin Dad Hat, Tommy Stone, One Size",Brazilian Deep Wave Bundles with Closure Virgin Human Hair Bundles with Closure 4×4 Lace Mixed Length Hair Bundles Natural Color for Black Women 100% Unprocessed Miss GAGA (16 18 20+14),"Little Live Pets Wrapples - Slap Bracelets - Shylo, Blue (28822)"
1,"Hatley Kids' Little Classic Rain Boots, Pink & Navy, 8 US Child","Very Fine Women's Delphine Ballroom Salsa Latin Tango Waltz Dance Shoe, Flesh, 9.5","COTTON DAY Kids Girls Fun Novelty Design Socks Bright Colors Gift Box Set (Animals duck swan fox rabbit, M: Shoe Size 10-13)","Jacobson Hat Company Men's Beard with Elastic, Grey, Adult, One Size",Anogol Hair Short Bob Wig Straight Cosplay For Kids With Bangs Pink Ombre Cosplay Wigs Synthetic Wig For Cosplay Party Halloween Without Hair Clip,"Mocasines para mujer, de la marca Ugg Australia, color Gris, talla 40","UGG Women's Ansley Moccasin, Light Grey, 9",Oud For Glory (Bade'e Al Oud) EDP Perfume By Lattafa: 3.4oz High End Special Premium Niche,Toddler Oompa Loompa Costume Charlie and the Chocolate Factory Costume for Kids 4T,"UGG Coquette Slipper, Wild Berry, Size 5",...,Disney Clara's Light-Up Dress Barbie Doll,"Funko Animation: Pop! One-Piece Collectors Set Series 3 - Sanji, Nico Robin, Usopp, Dq Doflamingo Toy","25MM Eyelashes Faux Mink, Calphdiar Fluffy Volume 5D False Eyelashes, Long Dramatic Lashes 20 pairs 4 styles","Guerlain Shalimar Eau De Parfum Spray for Women, 3 Ounce",Disney Minnie Mouse Polka Dot Pom Pom Baseball Cap with Bow Black,"Tommy Hilfiger Baby Girls 2 Pieces Legging Set, Red, 6-9 Months","Carhartt Women's Buffalo Cap, deep Wine, OFA","Pokemon TCG: Power Partnership Tag Team Tin, Lucario & Melmetal",Disney Junior Sheriff Callie's Wild West Callie Bean Bag Plush 8 Inches,"RED HEART Super Saver Jumbo Yarn, Aran"
2,Outee Rubber Kids Rain Boots,Cocktail Party Jazz 2: An Intoxicating Collection Of Instrumental Jazz,Unsigned #2 Orange Custom Stitched Baseball Jersey Size Men's XL New No Brands/Logos,Unsigned Sam Darnold New York 2019 Green Custom Stitched Football Jersey Size XL New No Brands/Logos,Making Friends (Mr. Rogers),Unsigned Rob Gronkowski New England White Custom Stitched Football Jersey Size XL New No Brands/Logos,Deer Stags Kids Matteo (Little Kid/Big Kid) Black 4 Big Kid,Un

In [124]:
gc.collect()

0

<font style="font-size:18px;color:green;" >
Chop-off the query tower & generate/get query embeddings
</font>

In [125]:
query_model = Model(inputs=input_query,
                         outputs=model.get_layer('normalize_query').output
                   )

In [126]:
plot_model(query_model)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [127]:
input_data_query = np.array(df_query_embedding[query_tower_cols])
query_embeddings = query_model.predict(input_data_query)
len(query_embeddings)

2212/2212 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


70756

In [128]:
df_query_embeddings_model = pd.DataFrame(query_embeddings, columns = [f'q{x}'for x in range(embedding_dim)] )

df_query_embeddings_model['query'] = df_query_embedding['query']
df_query_embeddings_model['qid'] = df_query_embedding['qid']

df_query_embeddings_model.head()

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q14,q15,q16,q17,q18,q19,q20,q21,query,qid
0,-0.183932,-0.522189,-0.093822,0.117379,0.381138,-0.189206,0.339233,-0.065131,-0.030373,-0.092502,...,0.145525,0.006570,0.069026,0.317311,-0.246756,-0.018882,-0.159031,0.049491,child proof cabinet locks,0
1,-0.106254,-0.265166,-0.030163,-0.194136,0.132043,0.078801,0.189409,0.068993,0.027913,-0.503185,...,0.099468,-0.163658,0.148606,0.006222,-0.030914,-0.009398,0.541846,-0.241984,ankle stockings for women sheer,1
2,-0.161229,-0.471039,-0.051871,0.082452,0.356398,-0.214856,0.341605,0.000508,-0.085393,0.221371,...,0.164226,0.051899,0.005513,0.326920,-0.247395,-0.089733,-0.030430,0.115255,gluten free snacks,2
3,-0.174280,-0.485012,-0.035860,-0.089558,0.326623,-0.107273,0.367897,0.076020,-0.065221,-0.047058,...,0.190186,-0.051757,0.076812,0.234473,-0.193143,-0.101202,0.394958,-0.056498,hair geow,3
4,-0.187444,-0.517309,-0.102042,0.113905,0.367569,-0.152792,0.313300,-0.102664,0.005389,-0.306372,...,0.126888,-0.020243,0.116896,0.275690,-0.223893,0.025363,-0.211131,-0.012674,"1 by one, amplified, outdoor hdtv antenna",4


##### We can analyse the quality of learnt query embeddings of 22-dimensions by doing an ANN search/similarity search

In [52]:
from annoy import AnnoyIndex
# qm = AnnoyIndex(embedding_dim, 'euclidean')
qm = AnnoyIndex(embedding_dim, 'dot')
mp_query_dict = {}

for ix,row in df_query_embeddings_model.iterrows():
    mp_query_dict[row['qid']] = row['query']
    
    key = int(row['qid'])
    vec = list(row[['q'+str(x) for x in list(range(embedding_dim))]])

    qm.add_item(key,vec)

In [53]:
qm.build(100)
qm.save('query_model.tree')

# qm = AnnoyIndex(embedding_dim,  'euclidean')
qm = AnnoyIndex(embedding_dim,  'dot')
qm.load('query_model.tree')

True

In [54]:
top_k = 20
mat = []
for ix,row in df_query_embeddings_model.iterrows():
    item = row['query']
    mat.append([item] + [mp_query_dict[x] for x in qm.get_nns_by_item(row['qid'], top_k+1)[1:]])
    
    if ix == 50:
        break
    
cols = ['query_id']
for i in range(top_k):
    cols += ['nearest_{}'.format(i+1)]

print(cols)

df_neighbors4 = pd.DataFrame(mat, columns = cols)

display(df_neighbors4.head(50))

['query_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


,query_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,child proof cabinet locks,water cooler,badge holder reel clip retractable,6 outlet grounded wall tap,metal straw stickers,non adhesive shelf liner,nonstick pots set,vinyl plank peel and stick,lip gloss tubes with wand empty,ortho home defense hose sprayer,...,hydro dip kit,ball pump,planter without drainage hole,tree free toilet paper,round vessel sink,stainless paper towel holder,grill stand,quick pitch shower kit,cat scratching cardboard refill,tub stopper
1,ankle stockings for women sheer,silk bathing suit cover up,smart wool socks womens medium,plus size knee high stockings for women,fluffy jackets for women,lace up wedge sandals for women,women black shirt extra small,denim jacket plus size oversized,tennis skirts without shorts,gauze skirts for women,...,corsette tops,bras without underwire,sequin dress long,knee high stockings for women plus size,blouses for women,mid calf boots for women,shirts for leggings,maxi slit skirt,women hiking boots,bootcut leggings for women
2,gluten free snacks,pickle rick ornament,old bay hot sauce,flavored waer,scented auto carpet shampoo,pumpkin pancake mold,clean and clear face wash,niagara starch spray,limes fresh organic,clarified butter not ghee,...,black seed oil for hair,bed head shampoo and conditioner,bitter apple spray for dogs,old dutch dill pickle chips,baby bottles 10 oz,washout hair dye,yerba mate powder mix,aqueon betta food,shampoo and conditioner for color treated hair,raw peanuts without skin
3,hair geow,champion sweatshirt,ducky,soccer goals for backyard,halloween coustume disney duo,cauliflower gnocchi,la roche posay sunscreen,dark brown long hair wig,diary of the wimpy kid books,laird mushroom,...,pokemon card binder,ugg slippers kids,oh boy baby bag,eye pads for puffy eyes,1 year old sandals,ventaire baby bottles,playpen guinea pig,capitals dog bandana,herbatint dark blonde,babybliss curling iron
4,"1 by one, amplified, outdoor hdtv antenna",outdoor speakers,lights,plates,re:zero figure,3 tier plant stand,fixit tool kit,7 x 5 acrylic sign holder landscape,solar flood lights outdoor dusk to dawn,fart glass bombs,...,echo input,magpul phone case,rc cars that go 100 mph,metal signs,jewelry stamping kit,bass guitar stickers,die cut decals,"brother replacement tape for label maker .47""",lights that sync with music,airplane toys
5,adidas sweats slim,michael connelly,james madison university football apparel,adidas pants,jeffrey epstein,nba youngboy vest,harlem godfather,alejandro junger,wwe toys elite nwo,marvel legends venom,...,cthulhu wars,super mario sunshine n64,spiderman cologne,greys anatomy yang,thunderbirds,marvel against humanity game,chris bachelder,punisher face mask,alex chiu,utah jazz jersey
6,beta sitosterol now,coffin nails occult,college scantrons,essence mascara,90s,day of the dead makeup,9th anniversary gifts,48 laws of power,probiotoc,star theme room decor,...,focus eyeshadow palette,patterns for college writing 13th edition,the old republic revan,day of the dead in the usa,killz,lantern press,astrology books,gifts for 6-year-old boys,manafier glass,nan madol
7,iphone 11 pro,samsung tab s4 keyboard case,phone screen protector iphone 7,iphone xs max phone case sonix,protector for iphone xr,strong screen protector iphone xr,samsunga70 phone,great wave phone case iphone xr,nexus 5x camera glass replacement,iphone adapter for iphone 7,...,apple ipad 9.7 case,cover samsung a20,screen protector iphone x,spectra s2 battery pack,uv-82 antenna,hp laptop red,new iphone 7 plus,usbc battery pack macbook pro,samsung ssd 1tb,screen protector for iphone xr
8,large training pads for dogs,light purple heart stickers,landtone,dimond studded dremel bits,hang bird feeder,back mirror for baby,zeirbah,bixlar,threadart embroidery designs usb,dodge ram hemi shirts,

<font style="font-size:18px;color:green;" >
Random Queries generated from Chat-Gpt for an ecommerce website
</font>

In [129]:
product_search_queries = [
    "Red dress for a wedding",
    "Best laptop under $1000",
    "Nike running shoes for women",
    "iPhone 13 reviews",
    "Men's winter jackets on sale",
    "Top-rated kitchen appliances",
    "Summer dresses for women",
    "Samsung TV specifications",
    "Gift ideas for anniversary",
    "XYZ brand headphones",
    "Women's shoes size 8",
    "Deals on smartphones",
    "Men's formal suits",
    "Fitness trackers with heart rate monitor",
    "Digital cameras with 4K video",
    "Wireless gaming mouse",
    "Outdoor camping gear",
    "Bluetooth earbuds under $50",
    "Designer handbags on clearance",
    "Gaming laptops with RTX graphics",
    "Organic skincare products",
    "Smartphones with long battery life",
    "Running shoes for flat feet",
    "Laptop deals for students",
    "Wireless noise-canceling headphones",
    "Digital cameras for beginners",
    "Best coffee makers under $50",
    "Latest fashion trends for summer",
    "Top-rated gaming keyboards",
    "Dining room furniture sets",
    "Winter coats for kids",
    "Men's watches with leather straps",
    "Home gym equipment for small spaces",
    "Women's handbags on sale",
    "Bluetooth speakers with waterproof features",
    "Outdoor patio furniture",
    "Affordable fitness trackers",
    "Cookware sets for induction cooktops",
    "Portable chargers for smartphones",
    "Desktop computers for gaming",
]
product_search_queries

['Red dress for a wedding',
 'Best laptop under $1000',
 'Nike running shoes for women',
 'iPhone 13 reviews',
 "Men's winter jackets on sale",
 'Top-rated kitchen appliances',
 'Summer dresses for women',
 'Samsung TV specifications',
 'Gift ideas for anniversary',
 'XYZ brand headphones',
 "Women's shoes size 8",
 'Deals on smartphones',
 "Men's formal suits",
 'Fitness trackers with heart rate monitor',
 'Digital cameras with 4K video',
 'Wireless gaming mouse',
 'Outdoor camping gear',
 'Bluetooth earbuds under $50',
 'Designer handbags on clearance',
 'Gaming laptops with RTX graphics',
 'Organic skincare products',
 'Smartphones with long battery life',
 'Running shoes for flat feet',
 'Laptop deals for students',
 'Wireless noise-canceling headphones',
 'Digital cameras for beginners',
 'Best coffee makers under $50',
 'Latest fashion trends for summer',
 'Top-rated gaming keyboards',
 'Dining room furniture sets',
 'Winter coats for kids',
 "Men's watches with leather straps",


In [130]:
gc.collect()

2784

In [ ]:

# from laserembeddings import Laser
# # Load LASER model
# laser = Laser()

In [131]:
import numpy as np

def reshape_array(input_array, d):
    k, _ = input_array.shape
    new_array = np.zeros((k, d))
    
    for i in range(k):
        for j in range(d):
            start_idx = j * (768// d)
            end_idx = (j + 1) * (768// d) if j < (d - 1) else 768
            chunk = input_array[i, start_idx:end_idx]
            new_array[i, j] = np.mean(chunk)
    
    return new_array

In [ ]:
# query_max_words = 10
# product_title_max_words = 50
# product_description_max_words = 20
# product_bullet_point_max_words = 50
# product_brand_max_words = 5
# product_color_max_words = 5

In [132]:
gc.collect()

0

In [133]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

In [134]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel  # Make sure to import these if not done already

# Define the model and tokenizer (assuming this is done earlier in your code)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Parameters
query_dim = 32
query_tower_cols = ['q' + str(x) for x in range(query_dim)]

# Assuming product_search_queries is defined
sentences = product_search_queries

# Encode the input sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Generate sentence embeddings
with torch.no_grad():
    output = model(**encoded_input)

# Extracting the embeddings
sentence_embeddings = output.last_hidden_state[:, 0, :]

# Convert to NumPy array
sentence_embeddings_np = sentence_embeddings.numpy()  # Convert to NumPy array

# Reshape the embeddings if necessary
# Here we assume the embeddings shape is (batch_size, hidden_size)
# If hidden_size is different from query_dim, we may need to handle this.
if sentence_embeddings_np.shape[1] != query_dim:
    # You can either reshape or take a subset of the embeddings, e.g.:
    # Assuming hidden_size is 768, we can take the first `query_dim` features
    # Only do this if you're sure it makes sense for your application.
    sentence_embeddings_np = sentence_embeddings_np[:, :query_dim]  # Take first query_dim columns

# Create a DataFrame with the embeddings
df_query_test_actual = pd.DataFrame(sentence_embeddings_np)
df_query_test_actual.columns = query_tower_cols
df_query_test_actual['query'] = product_search_queries

# Display the DataFrame
df_query_test_actual.head(10)


,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q23,q24,q25,q26,q27,q28,q29,q30,q31,query
0,-0.126827,-0.398998,0.021677,-0.014519,0.031822,-0.021648,0.155635,0.413161,-0.284341,-0.097350,...,-0.046889,0.184771,-0.230426,-0.047964,0.049216,0.207972,0.058643,0.068970,-0.016755,Red dress for a wedding
1,-0.602774,-0.190369,0.072234,-0.029794,-0.087956,-0.080643,0.166529,0.440694,-0.012971,-0.203527,...,-0.023467,-0.093033,-0.024835,0.057742,-0.003708,-0.042907,0.109425,-0.009318,0.225178,Best laptop under $1000
2,-0.119987,-0.253422,-0.006892,-0.043434,-0.075241,0.045696,0.244303,0.247582,-0.345697,-0.024551,...,-0.013493,0.051530,-0.066295,0.258473,0.001565,0.136067,0.151185,0.158018,0.101633,Nike running shoes for women
3,-0.316359,-0.122843,0.095608,-0.071883,0.052767,0.026433,0.179699,0.187807,-0.091912,-0.137030,...,-0.056003,-0.172856,-0.083697,0.102127,-0.044567,-0.018378,-0.004959,-0.197404,0.304808,iPhone 13 reviews
4,-0.281021,-0.195129,-0.045723,-0.196411,0.040131,-0.250816,0.063603,0.324732,-0.347106,-0.044500,...,-0.096691,0.290675,-0.220582,0.249059,-0.068226,0.150535,0.242574,0.033169,0.137612,Men's winter jackets on sale
5,-0.407927,-0.098069,0.085701,-0.166961,0.007280,-0.009624,0.075347,0.212776,-0.105331,-0.024146,...,-0.098583,-0.091657,0.067742,-0.054591,-0.083030,0.075008,0.252140,0.053241,0.209900,Top-rated kitchen appliances
6,-0.183327,-0.335780,-0.062052,-0.104124,-0.006858,-0.048101,0.100690,0.371781,-0.395992,-0.049306,...,-0.029134,0.051423,-0.310863,0.143242,0.012755,0.244394,0.131120,-0.028037,0.026205,Summer dresses for women
7,-0.288985,-0.106961,0.078907,-0.249443,-0.008073,-0.055646,0.144325,0.143418,-0.087818,-0.079625,...,-0.265872,0.005742,-0.097310,0.122056,-0.026557,0.197636,0.134063,-0.123527,0.182986,Samsung TV specifications
8,-0.133388,-0.185080,0.086916,-0.006108,-0.039839,-0.112558,0.007224,0.369299,-0.232651,-0.154225,...,-0.027931,-0.087539,-0.078331,0.067143,-0.020722,0.156039,0.087367,0.079268,0.097897,Gift ideas for anniversary
9,-0.151111,-0.040483,0.164673,-0.089193,-0.102079,0.094723,0.108064,0.120575,-0.032664,-0.092742,...,-0.038755,-0.175713,0.048173,0.108835,-0.010038,0.212372,0.067934,0.058805,0.318427,XYZ brand headphones


import torch
import numpy as np
import pandas as pd
query_dim = 32

# Adjusting the length of query_tower_cols to match the expected number of columns
query_tower_cols = ['q' + str(x) for x in range(query_dim)]

# Assuming the rest of your code remains the same
sentences = product_search_queries
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Generate sentence embeddings
with torch.no_grad():
    output = model(**encoded_input)
sentence_embeddings = output.last_hidden_state[:, 0, :]

sentence_embeddings = reshape_array(np.array(sentence_embeddings), query_dim)
    
df_query_test_actual = pd.DataFrame(sentence_embeddings)
df_query_test_actual.columns = query_tower_cols
df_query_test_actual['query'] = product_search_queries

df_query_test_actual.head(10)


In [135]:
sentence_embeddings

tensor([[-0.1268, -0.3990,  0.0217,  ..., -0.1503,  0.1249,  0.2381],
        [-0.6028, -0.1904,  0.0722,  ..., -0.1290,  0.2753,  0.1426],
        [-0.1200, -0.2534, -0.0069,  ..., -0.1259, -0.0127,  0.1974],
        ...,
        [-0.1850, -0.1853,  0.1003,  ..., -0.1719,  0.0035,  0.3255],
        [-0.3002, -0.3848, -0.0160,  ..., -0.2073,  0.0560,  0.1652],
        [-0.4016, -0.2944, -0.0219,  ..., -0.3506,  0.1449,  0.3255]])

In [136]:
input_data_query = np.array(df_query_test_actual[query_tower_cols]),
query_embeddings = query_model.predict(input_data_query)
len(query_embeddings)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


40

In [137]:
cols = [f'q{x}'for x in range(embedding_dim)]

df_query_test = pd.DataFrame(query_embeddings, columns = cols)
df_query_test['query'] = product_search_queries

df_query_test.head(50)

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q13,q14,q15,q16,q17,q18,q19,q20,q21,query
0,-0.132895,-0.403266,-0.035977,-0.075138,0.283294,-0.129498,0.338632,0.130227,-0.098393,0.260988,...,0.144819,0.177191,-0.026226,-0.019128,0.268784,-0.187661,-0.146383,0.451732,0.048354,Red dress for a wedding
1,-0.033153,-0.077917,0.060643,-0.279374,0.017460,0.066223,0.135845,0.201606,-0.065764,0.077672,...,-0.076086,0.110850,-0.086870,0.020937,-0.039246,0.010665,-0.129657,0.830338,-0.135236,Best laptop under $1000
2,-0.120143,-0.337908,0.021703,-0.153118,0.229951,-0.095107,0.309274,0.153179,-0.112409,0.261507,...,0.100056,0.186626,-0.015584,0.005542,0.176693,-0.148402,-0.160967,0.591398,-0.010124,Nike running shoes for women
3,-0.121304,-0.366070,-0.006842,0.000978,0.281883,-0.192283,0.310078,0.085426,-0.124757,0.455585,...,0.206438,0.169899,0.061920,-0.049910,0.281980,-0.208797,-0.149051,0.224923,0.135024,iPhone 13 reviews
4,-0.140101,-0.396535,0.002519,-0.118813,0.273253,-0.114577,0.336696,0.123528,-0.102478,0.194775,...,0.124634,0.192206,-0.018104,0.019218,0.213236,-0.174237,-0.143287,0.521338,-0.003554,Men's winter jackets on sale
5,-0.109255,-0.333308,0.002732,0.008689,0.263414,-0.196958,0.289777,0.083793,-0.130229,0.511983,...,0.210157,0.162788,0.077376,-0.063954,0.273744,-0.202132,-0.151381,0.204267,0.154886,Top-rated kitchen appliances
6,-0.139105,-0.407551,-0.013347,-0.054511,0.295380,-0.158573,0.340638,0.105534,-0.112253,0.313243,...,0.172266,0.186540,0.014397,-0.013995,0.265918,-0.202505,-0.146908,0.371540,0.066050,Summer dresses for women
7,-0.132825,-0.400503,-0.028785,0.056784,0.311631,-0.213559,0.315443,0.042116,-0.109981,0.398341,...,0.232300,0.161852,0.068774,-0.041184,0.311617,-0.229284,-0.122679,0.074436,0.152865,Samsung TV specifications
8,-0.119375,-0.342934,0.023031,-0.068996,0.256301,-0.163189,0.309923,0.117394,-0.133687,0.439579,...,0.171345,0.185684,0.049332,-0.032194,0.229157,-0.186753,-0.164888,0.388702,0.083727,Gift ideas for anniversary
9,-0.150229,-0.438679,-0.026734,0.056231,0.336207,-0.216350,0.333677,0.021438,-0.103841,0.316474,...,0.235327,0.171796,0.067048,-0.011231,0.310596,-0.239294,-0.110934,0.051738,0.127461,XYZ brand headphones


In [139]:
results = []
top_k = 20

desc = []
for i in range(top_k):
    desc.append(f'product_title_{(i+1)}')
desc = ['query'] + desc

for ix,row in df_query_test.iterrows():
    qvec = [f'q{x}' for x in list(range(embedding_dim))]
    qvec = row[qvec]
    similar_vector_ids,similar_distances = pm.get_nns_by_vector(qvec, top_k, include_distances=True)
    similar_vector_product_title = [mp_product_dict[x] for x in similar_vector_ids]
    results.append([row['query']] + similar_vector_product_title)

df_results = pd.DataFrame(results, columns =  desc)

df_results.head(50)

,query,product_title_1,product_title_2,product_title_3,product_title_4,product_title_5,product_title_6,product_title_7,product_title_8,product_title_9,...,product_title_11,product_title_12,product_title_13,product_title_14,product_title_15,product_title_16,product_title_17,product_title_18,product_title_19,product_title_20
0,Red dress for a wedding,"Cala Hot and Cold Eye Pads - Relieves Puffiness and Tension, Watermelon (69163)","Pacifica Beauty, Pink Nudes Mineral Eyeshadow Palette, 10 Neutral Shades, For Natural or Smoky Eye Look, Eye Makeup, Longwearing and Blendable, Infused with Coconut Water, 100% Vegan and Cruelty Free","Almay Thickening Mascara with Aloe and Vitamin B5, Hypoallergenic, Cruelty Free, -Fragrance Free, Ophthalmologist Tested, 401 Blackest Black, 0.26 Fl Oz (Pack of 1)","L'Oreal Paris Excellence Creme Permanent Hair Color, 4 Dark Brown, 100 percent Gray Coverage Hair Dye, Pack of 1","Baby Buddy Natural Baby Bath Sponge 4” High Quality Soft Grass Sea Sponge Soft on Baby’s Tender Skin, Biodegradable, Hypoallergenic, Absorbent Natural Sea Sponge, 1 Pack","Champú Matizador Rubio con Pigmentos Violetas Para Conseguir Tonos Plateados - Adiós al Amarillo: Revitaliza el Cabello Rubio Teñido, Decolorado y con Mechas - Sin Sulfatos - 237 ml","Purple Shampoo for Blonde Hair: Blonde Shampoo Eliminates Brassy Yellow Tones- Lightens Blonde, Platinum, Ash, Silver and Grays- Paraben & Sulfate Free Toner- Revitalize Bleached & Highlighted Hair","Maybelline TattooStudio Longwear Waterproof Eyebrow Gel Makeup for Fully Defined Brows, Spoolie Applicator Included, Lasts Up To 2 Days, Black Brown, 0.23 Fl Oz (Pack of 1)","Clairol Root Touch-Up by Nice'n Easy Permanent Hair Dye, 7 Dark Blonde Hair Color, 2 Count",...,"Reb'L Fleur Eau de Parfum Spray for Women, Mini, 1 Fluid Ounce","HALLS Relief Variety Pack Honey Lemon and Black Cherry Sugar Free Cough Drops, 6 Packs of 25 Drops (150 Total Drops)","Head & Shoulders Royal Oils Moisture Shampoo and Conditioner Bundle, 27 fl oz, Kit (Package May Vary)","Baby Buddy Natural Baby Bath Sponge 4"" Ultra Soft Premium Sea Wool Sponge Soft On Baby's Tender Skin, Biodegradable, Hypoallergenic, Absorbent Natural Sea Sponge, 1 Pack, Brown, 4""","Flowerbomb by Viktor & Rolf For Women, Eau de Parfum, 1.7-Ounce Spray","Collins Consumables Stemmed Royal Anne Cherries Premium Garnish for Cocktails Martinis, Manhattan, Old Fashioned, Desserts, Drinks, 10 Ounce, Black","Caron One Pound Yarn, Medium Gray Mix","Aussie Mega Hairspray, with Jojoba Oil & Sea Kelp, Flexible Hold, 17 fl oz, Triple Pack","Crocodile Creek - Dogs - Rubber Playground Ball, 5"", for Kids Ages 3 Years & Up, Purple, 1 EA","L'Oreal Men Expert Vitalift Anti-Wrinkle & Firming Face Moisturizer with Pro-Retinol, Face Moisturizer for Men, Beard and Skincare for Men, 1.6 oz"
1,Best laptop under $1000,"Signature by Levi Strauss & Co. Gold Label Women's Mid-Rise Bermuda Shorts, Stormy Sky, 14","Signature by Levi Strauss & Co. Gold Label Women's Totally Shaping Pull-On Skinny Jeans, Harmony, 18","Signature by Levi Strauss & Co Women's Totally Shaping Pull On Skinny Jeans, Harmony, 10 Short","Baikea Womens Buffalo Plaid Tops, Raglan Sleeves Christmas Shirt Long Sleeve Checkered Tunic Tops Round Neck Holiday Sweatshirt Red Black M","La Blanca Women's Standard Bandeau Tankini Swimsuit Top, Black//Island Goddess, 8","O'Neill Wetsuits Women's Basic Skins Short Sleeve Rash Tee, Heathered Pink - SM","La Blanca Women's Standard Underwire Double Strappy Back One Piece Swimsuit, Watermelon//Island Goddess, 4","Signature by Levi Strauss & Co. Gold Label Boys' Skinny Fit Jeans, gothic, 14","Signature by Levi Strauss & Co. Gold Label Women's Plus Mid Rise Slim Boyfriend Jeans, stormy sky Canada, 20",...,Pink Queen Women's 2 Pieces Bandeau Bikini Swimsuits High Waist Brown Leopard L,"Signature by Levi Strauss & Co. Gold Label Women's Totally Shaping Pull-On Skinny Jeans, Harmony-Waterless, 10","Signature by Levi Strauss & Co. Gold Label Women's Mid-Rise